# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 12 2023 10:39AM,264552,15,9565708,Lonza,Released
1,Jun 12 2023 10:26AM,264549,21,9565688,"NBTY Global, Inc.",Released
2,Jun 12 2023 10:11AM,264548,18,EMERSA-9565682,"Emersa Waterbox, LLC",Released
3,Jun 12 2023 10:10AM,264547,15,30017174,"Alliance Pharma, Inc.",Released
4,Jun 12 2023 10:10AM,264547,15,30017175,"Alliance Pharma, Inc.",Released
5,Jun 12 2023 10:10AM,264547,15,30017176,"Alliance Pharma, Inc.",Released
6,Jun 12 2023 10:10AM,264547,15,30017201,"Alliance Pharma, Inc.",Released
7,Jun 12 2023 10:10AM,264547,15,30017202,"Alliance Pharma, Inc.",Released
8,Jun 12 2023 10:10AM,264547,15,30017203,"Alliance Pharma, Inc.",Released
9,Jun 12 2023 10:10AM,264547,15,30017204,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,264546,Released,1
14,264547,Released,31
15,264548,Released,1
16,264549,Released,1
17,264552,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264546,NaN,1.0
264547,NaN,31.0
264548,NaN,1.0
264549,NaN,1.0
264552,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264530,15.0,5.0
264533,1.0,0.0
264534,0.0,43.0
264535,0.0,1.0
264537,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264530,15,5
264533,1,0
264534,0,43
264535,0,1
264537,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264530,15,5
1,264533,1,0
2,264534,0,43
3,264535,0,1
4,264537,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264530,15,5
1,264533,1,
2,264534,,43
3,264535,,1
4,264537,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 12 2023 10:39AM,264552,15,Lonza
1,Jun 12 2023 10:26AM,264549,21,"NBTY Global, Inc."
2,Jun 12 2023 10:11AM,264548,18,"Emersa Waterbox, LLC"
3,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc."
34,Jun 12 2023 10:08AM,264546,18,"Emersa Waterbox, LLC"
35,Jun 12 2023 10:03AM,264545,21,"NBTY Global, Inc."
36,Jun 12 2023 9:58AM,264544,10,Alikai Health
52,Jun 12 2023 9:56AM,264543,10,"Citieffe, Inc."
54,Jun 12 2023 9:54AM,264542,15,"Alliance Pharma, Inc."
81,Jun 12 2023 9:49AM,264540,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 12 2023 10:39AM,264552,15,Lonza,,1
1,Jun 12 2023 10:26AM,264549,21,"NBTY Global, Inc.",,1
2,Jun 12 2023 10:11AM,264548,18,"Emersa Waterbox, LLC",,1
3,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc.",,31
4,Jun 12 2023 10:08AM,264546,18,"Emersa Waterbox, LLC",,1
5,Jun 12 2023 10:03AM,264545,21,"NBTY Global, Inc.",,1
6,Jun 12 2023 9:58AM,264544,10,Alikai Health,,16
7,Jun 12 2023 9:56AM,264543,10,"Citieffe, Inc.",,2
8,Jun 12 2023 9:54AM,264542,15,"Alliance Pharma, Inc.",5,22
9,Jun 12 2023 9:49AM,264540,19,"AdvaGen Pharma, Ltd",2,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 10:39AM,264552,15,Lonza,1,
1,Jun 12 2023 10:26AM,264549,21,"NBTY Global, Inc.",1,
2,Jun 12 2023 10:11AM,264548,18,"Emersa Waterbox, LLC",1,
3,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc.",31,
4,Jun 12 2023 10:08AM,264546,18,"Emersa Waterbox, LLC",1,
5,Jun 12 2023 10:03AM,264545,21,"NBTY Global, Inc.",1,
6,Jun 12 2023 9:58AM,264544,10,Alikai Health,16,
7,Jun 12 2023 9:56AM,264543,10,"Citieffe, Inc.",2,
8,Jun 12 2023 9:54AM,264542,15,"Alliance Pharma, Inc.",22,5
9,Jun 12 2023 9:49AM,264540,19,"AdvaGen Pharma, Ltd",10,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 10:39AM,264552,15,Lonza,1,
1,Jun 12 2023 10:26AM,264549,21,"NBTY Global, Inc.",1,
2,Jun 12 2023 10:11AM,264548,18,"Emersa Waterbox, LLC",1,
3,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc.",31,
4,Jun 12 2023 10:08AM,264546,18,"Emersa Waterbox, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 10:39AM,264552,15,Lonza,1.0,NaN
1,Jun 12 2023 10:26AM,264549,21,"NBTY Global, Inc.",1.0,NaN
2,Jun 12 2023 10:11AM,264548,18,"Emersa Waterbox, LLC",1.0,NaN
3,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc.",31.0,NaN
4,Jun 12 2023 10:08AM,264546,18,"Emersa Waterbox, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 10:39AM,264552,15,Lonza,1.0,0.0
1,Jun 12 2023 10:26AM,264549,21,"NBTY Global, Inc.",1.0,0.0
2,Jun 12 2023 10:11AM,264548,18,"Emersa Waterbox, LLC",1.0,0.0
3,Jun 12 2023 10:10AM,264547,15,"Alliance Pharma, Inc.",31.0,0.0
4,Jun 12 2023 10:08AM,264546,18,"Emersa Waterbox, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1322688,67.0,15.0
15,793641,54.0,5.0
18,793629,3.0,0.0
19,529073,10.0,3.0
21,529094,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1322688,67.0,15.0
1,15,793641,54.0,5.0
2,18,793629,3.0,0.0
3,19,529073,10.0,3.0
4,21,529094,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,67.0,15.0
1,15,54.0,5.0
2,18,3.0,0.0
3,19,10.0,3.0
4,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,67.0
1,15,Released,54.0
2,18,Released,3.0
3,19,Released,10.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,21
Status,,,,,
Executing,15.0,5.0,0.0,3.0,0.0
Released,67.0,54.0,3.0,10.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,21
0,Executing,15.0,5.0,0.0,3.0,0.0
1,Released,67.0,54.0,3.0,10.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,21
0,Executing,15.0,5.0,0.0,3.0,0.0
1,Released,67.0,54.0,3.0,10.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()